In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import numpy as np
import torch
import crocoddyl
from utils import m2a
import time
import math

import numpy as np
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
import numdifftools as nd
from jacobian import JacobianReg
from tqdm import tqdm

In [4]:
def state_data(size:int = 1000, theta:float = 0.):
    """
    @params:
        1: size  = size of the dataset
        2: theta = float, between 0, 1
    
    Returns xtrain, ytrain.
    Returns data in the form of x --> V(x)
    
    """

    _xtrain = []
    _ytrain = []

    
    for _ in range(size):
        # Generate random starting configuration
        xyz = [np.random.uniform(-2.1, 2.1), 
               np.random.uniform(-2.1, 2.1),
               theta]
        
        
        model = crocoddyl.ActionModelUnicycle()
        T = 30
        model.costWeights = np.matrix([1,1]).T
        
        problem = crocoddyl.ShootingProblem(m2a(xyz).T, [ model ] * T, model)
        ddp = crocoddyl.SolverDDP(problem)
        ddp.solve([], [], 1000)
        xs = m2a(ddp.us).flatten()
        _ytrain.append(xs)        
        _xtrain.append(xyz)
        
    xtrain = torch.tensor(_xtrain, dtype = torch.float32)
    ytrain = torch.tensor(_ytrain, dtype = torch.float32)
        
    return xtrain, ytrain

class FeedForwardNet(nn.Module):
    def __init__(self, 
                 input_features,
                 output_features,
                 n_hidden_units = 512,
                 activation = 'relu'):
        
        
        """
        A two hidden layered neural network.
        
        @params:
            1: input_features = number of input features of the dataset
            2: output_features = number of output features of the dataset
            3: n_hidden_units = number of hidden units in hidden layer
            4: activation = either relu or tanh
            
            
        @returns:
            A fully connected feedforward network 
        
        """       
        
        super(FeedForwardNet, self).__init__()
        
        if activation == 'relu':
            self.activation = F.relu
            
        else:
            self.activation = torch.tanh
        # Structure
        self.fc1 = nn.Linear(input_features, n_hidden_units)
        self.fc2 = nn.Linear(n_hidden_units, n_hidden_units)
        self.fc3 = nn.Linear(n_hidden_units, output_features)
        
        # Initialization protocol
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.xavier_uniform_(self.fc3.weight)
        
      
        self.device = torch.device('cpu')
        self.to(self.device)

    def forward(self, x):     
        
        x1 = self.activation(self.fc1(x)) 
        x2 = self.activation(self.fc2(x1)) 
        x3 = self.fc3(x2) 
        return x3

xtrain, ytrain = state_data(1000)
dataset = torch.utils.data.TensorDataset(xtrain,ytrain)
dataloader = torch.utils.data.DataLoader(dataset, batch_size = 1000) # DataGenerator


# Generate a Neural Net
net = FeedForwardNet(input_features = xtrain.shape[1], 
                     output_features = ytrain.shape[1],
                     n_hidden_units = 256)

# set the net to training mode
net = net.float()
net.train()


# Define the loss function for optimizer
criteria = torch.nn.MSELoss(reduction='sum')
optimizer = optim.Adam(net.parameters(), lr=1e-3, weight_decay = 0.1)   

# Jacobian regularization
reg = JacobianReg() 
lambda_JR = 0.01 

t0 = time.time()    
# Training    
for epoch in tqdm(range(10000)):        
    for data, target in dataloader:   
        data.requires_grad=True
        optimizer.zero_grad()
        output = net.forward(data)          
        loss_super = criteria(output, target)
        R = reg(data, output)                            # Jacobian regularization
        loss = loss_super + lambda_JR*R                  # full loss
        loss.backward()
        optimizer.step()                                      

print('Training lasted = %.0f seconds' % (time.time()-t0))        

100%|██████████| 10000/10000 [12:12<00:00, 13.65it/s]

Training lasted = 732 seconds


In [6]:
xtest, ytest = state_data(1000)
net.eval()

with torch.no_grad():
    y_pred = net(xtest)
    
mean_squared_error = (ytest - y_pred) **2
    
print(f"Mean Squared Error during testing is {torch.mean(mean_squared_error)}") 

torch.save(net, "ControlNet.pth")

Mean Squared Error during testing is 0.0007638324750587344


/home/amit/.local/lib/python3.6/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type FeedForwardNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
